In [1]:
from pprint import pprint
from collections import namedtuple
import pickle

In [2]:
word_info=namedtuple("word_info","id word pos head rel".split())

In [3]:
data=pickle.load(open("./data.pkl","rb"))

In [4]:
data[31]

[word_info(id=0, word='He', pos='PRON', head=8, rel='nsubj'),
 word_info(id=1, word='is', pos='AUX', head=8, rel='cop'),
 word_info(id=2, word='no', pos='ADV', head=3, rel='advmod'),
 word_info(id=3, word='longer', pos='ADV', head=6, rel='advmod'),
 word_info(id=4, word='in', pos='ADP', head=8, rel='case'),
 word_info(id=5, word='the', pos='DET', head=8, rel='det'),
 word_info(id=6, word='dominant', pos='ADJ', head=8, rel='amod'),
 word_info(id=7, word='Shiite', pos='ADJ', head=8, rel='amod'),
 word_info(id=8, word='list', pos='NOUN', head=-1, rel='root'),
 word_info(id=9, word=',', pos='PUNCT', head=8, rel='punct'),
 word_info(id=10, word='the', pos='DET', head=13, rel='det'),
 word_info(id=11, word='United', pos='PROPN', head=13, rel='compound'),
 word_info(id=12, word='Iraqi', pos='PROPN', head=13, rel='compound'),
 word_info(id=13, word='Alliance', pos='PROPN', head=8, rel='appos'),
 word_info(id=14, word=',', pos='PUNCT', head=18, rel='punct'),
 word_info(id=15, word='and', pos='C

In [5]:
def shift(stack,buffer,actions):
    stack.append(buffer.pop(0))
    actions.append("shift")
    print("{:10} [{:>20}] [{:40}]".format("shift",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    return stack,buffer,actions

In [6]:
def swap(stack,buffer,actions):
    buffer.insert(0,stack.pop(-2))
    actions.append("swap")
    print("{:10} [{:>20}] [{:40}]".format("swap",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    return stack,buffer,actions

In [7]:
def left_r(stack,actions):
    
    _=stack.pop(-2)
    actions.append("left-reduce")
    print("{:15} [{:>20}] [{:40}]".format("left-r",
                                        " ".join([wi.word for wi in stack]),
                                        " ".join([wi.word for wi in buffer])))
    return stack,actions
    

In [8]:
def right_r(stack,actions):
    
    _=stack.pop(-1)
    actions.append("right-reduce")
    print("{:15} [{:>20}] [{:40}]".format("right-r",
                                " ".join([wi.word for wi in stack]),
                                " ".join([wi.word for wi in buffer])))
    return stack,actions

In [9]:
def next_child(word,buffer):
    for wi in buffer:
        if word.id==wi.head:
            return wi.id
    return None

In [ ]:
actions = []
outputs = []

for sent in data[31:]:
    stack = [word_info(-1, "ROOT", "ROOT", None, None)]
    buffer = sent[:]

    while len(buffer) > 0:
        if len(stack)==1:
            stack, buffer, actions=shift(stack, buffer, actions)
        else:
            
            if len(stack)==2:
                head=stack[-1].head
                child=next_child(stack[-1],buffer)
                print("head: {}\tchild: {}".format(head,child))
                if not child:
                    if head==buffer[0].head or head==buffer[0].id:
                        stack, buffer, actions=shift(stack, buffer, actions)
                    else:
                        stack, buffer, actions=shift(stack, buffer, actions)
                        stack, buffer, actions=shift(stack, buffer, actions)
                        stack, buffer, actions=swap(stack, buffer, actions)
                else:
                    while buffer[0].id==child:
                        stack, buffer, actions=shift(stack, buffer, actions)
                    while stack[-1].head==stack[-2].id:
                        stack, buffer, actions=shift(stack, buffer, actions)
            else:
                head_1=stack[-1].head
                child_1=next_child(stack[-1],buffer)
                head_2=stack[-2].head
                child_2=next_child(stack[-2],buffer)
                print("head_1: {}\tchild_1: {}\thead_2: {}\tchild_2: {}".format(head_1,child_1,head_2,child_2))
                if not child_1 and not child_2:
                    if head_1==stack[-2].id:
                        stack,actions=right_r(stack,actions)
                    elif head_2==stack[1].id:
                        stack,actions=left_r(stack,actions)
                    else:
                        if head_1==buffer[0].head or head_1==buffer[0].id:
                            stack, buffer, actions=shift(stack, buffer, actions)
                        else:
                            stack, buffer, actions=shift(stack, buffer, actions)
                            stack, buffer, actions=shift(stack, buffer, actions)
                            stack, buffer, actions=swap(stack, buffer, actions)
                elif child_2:
                    pass
                else:
                    while buffer[0].id==child_1:
                        stack, buffer, actions=shift(stack, buffer, actions)
                    while stack[-1].head==stack[-2].id:
                        stack, buffer, actions=shift(stack, buffer, actions)
            
            
        input()
        
    while len(stack)>1:
        if stack[-2].head == stack[-1].id:
                    stack, actions = left_r(stack, actions)
        elif stack[-2].id == stack[-1].head:
                    stack, actions = right_r(stack, actions)
        input()
    outputs.append(actions)
    actions = []

shift      [             ROOT He] [is no longer in the dominant Shiite list , the United Iraqi Alliance , and wo n't have many seats in the new parliament .]



''

head: 8	child: None
shift      [          ROOT He is] [no longer in the dominant Shiite list , the United Iraqi Alliance , and wo n't have many seats in the new parliament .]



''

head_1: 8	child_1: None	head_2: 8	child_2: None
shift      [       ROOT He is no] [longer in the dominant Shiite list , the United Iraqi Alliance , and wo n't have many seats in the new parliament .]
shift      [ROOT He is no longer] [in the dominant Shiite list , the United Iraqi Alliance , and wo n't have many seats in the new parliament .]
swap       [   ROOT He is longer] [no in the dominant Shiite list , the United Iraqi Alliance , and wo n't have many seats in the new parliament .]


In [11]:
pickle.dump(outputs,open("parsing_sequences.pkl","wb"))